In [1]:
import copy
import re
from collections import defaultdict

import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import numpy as np

# Topics

In [2]:
topics = pd.read_csv("../topic_words.csv.zip")
topics.head()

,meta,topic,weight,type,word
0,env,Климат,5,0,ch4
1,env,Климат,5,0,co2
2,env,Климат,5,0,ghg protocol
3,env,Климат,5,0,scope 1
4,env,Климат,5,0,scope 2


In [3]:
topics["topic"].value_counts()

Лидерство                          274
Энергия                            182
Риски                              177
Климат                             175
Социальные инвестиции              171
Обучение и развитие                160
Качество и безопасность продукт    158
Безопасность и охрана труда        145
Коренные народы                    126
Биоразнообразие                    118
Благотворительность                117
Вода                               114
Маркетинг, продажи, реклама        107
Оплата труда                       107
Отходы                             100
Воздух                              93
Отчетность и прозрачность           93
Ценовая политика                    86
Закупки и антикоррупция             85
Сервис и коммуникация               83
Рекультивация земель                77
Удовлетворенность потребителей      75
Профсоюз и Коллективный договор     73
Волонтерство                        71
Экологичность продукта              66
Доступность              

In [4]:
re_dict = {}
re_advanced_dict = {}
sheets_dict = {}
sheets_advanced_dict = {}
base_dict = {}
advanced_dict = {}
importance_dict = {}

for idx, row in topics.iterrows():
    if not sheets_dict.get(row["topic"], False):
        sheets_dict[row["topic"]] = {}
        sheets_advanced_dict[row["topic"]] = {}
        base_dict[row["topic"]] = {}
        advanced_dict[row["topic"]] = {}
    if row["word"].find(",") != -1:
        continue
    sheets_dict[row["topic"]][row["word"]] = row["weight"]  # row["type"]
    if row["type"] == 1:
        sheets_advanced_dict[row["topic"]][row["word"]] = row["weight"]
        advanced_dict[row["topic"]][row["word"]] = 0
    base_dict[row["topic"]][row["word"]] = 0
    importance_dict[row["topic"]] = 0

In [5]:
for key, val in sheets_dict.items():
    re_dict[key] = re.compile("((?<!\w)" + "(?!\w)|(?<!\w)".join(val.keys()) + "(?!\w))")

In [6]:
for key, val in sheets_advanced_dict.items():
    re_advanced_dict[key] = re.compile("((?<!\w)" + "(?!\w)|(?<!\w)".join(val.keys()) + "(?!\w))")

# Expert

In [7]:
regex_expert = re.compile("(" + "(?!\w)|".join(topics["word"].unique()) + "(?!\w))")

In [8]:
label2idx = {topic: i for i, topic in enumerate(sorted(topics["topic"].unique()))}
idx2label = {i: topic for i, topic in enumerate(sorted(topics["topic"].unique()))}
idx2word = {i: word for i, word in enumerate(topics["word"].tolist())}
word2idx = defaultdict(list)

for i, word in enumerate(topics["word"].tolist()):
    word2idx[word].append(i)

In [9]:
topics_vectors = np.zeros((topics["topic"].nunique(), topics.shape[0]), dtype=int)

for topic, topic_words in topics.groupby("topic")["word"].agg(list).items():
    for word in topic_words:
        for idx in word2idx[word]:
            topics_vectors[label2idx[topic], idx] = 1

In [10]:
matrix = np.zeros(topics_vectors.shape)
matrix.shape

(40, 3662)

In [11]:
def fit(vectors, labels):
    for vector, label in zip(vectors, labels):
        for i in range(matrix.shape[0]):
            if i == label2idx[label]:
                matrix[i] += vector
            else:
                matrix[i] -= vector


def test(vectors, labels):
    total = 0
    for vector, label in zip(vectors, labels):
        res = np.argmax(vector @ matrix.T)
        total += label == idx2label[res]
        if label != idx2label[res]:
            print("expected", label, "result", idx2label[res])
    print("accuracy", total / len(labels))

In [12]:
fit(topics_vectors, list(sorted(topics["topic"].unique())))
test(topics_vectors, list(sorted(topics["topic"].unique())))

accuracy 1.0


In [13]:
# text = "использовать передовой зарубежный достижение первостроитель магнитка скоро сам стать творец новый уникальный опыт ведь история такой грандиозный промышленный объект создаваться столь сжатый срок впервые мировой практика разработать метод добыча железный руда открытый карьер это смелый технический новшество год позволить достигнуть магнитогорский рудник выработка млн так руда составлять весь железный руда ссср"
text = "профильный предприятие газпром нефть внедрить грейд который позволять установить единый предприятие чёткий прозрачный принцип вознаграждение обучение оценка персонал в связь расширение география бизнес разработать внедряться единый концепция мобильный персонал кадровый база реализация крупный проект число зарубежный концепция определять порядок направление сотрудник работа регион россия граница перечень порядок предоставление компенсация льгота который позволять дополнительно мотивировать перемещать сотрудник организационный структура управление персонал включать департамент работа персонал корпоративный центр входить блок организационный вопрос управление работа персонал производственный блок подразделение работа персонал дзо этот структурный подразделение каждый свой уровень отвечать реализация стратегический программа развитие кадровый потенциал компания текущий обеспечение деятельность весь направление управление персонал"

print(text)
arr = []
for key, regex in re_dict.items():
    vector = np.zeros(matrix.shape[1])
    words = regex.findall(text)
    paragraph_dict = copy.deepcopy(base_dict)
    importance = 0
    for word in words:
        paragraph_dict[key][word] = sheets_dict[key][word]
        importance += sheets_dict[key][word] > 1
        vector[word2idx[word]] = 1

    dist = cosine_similarity(X=[list(sheets_dict[key].values())], Y=[list(paragraph_dict[key].values())])[0][0]
    arr.append((key, dist, importance, words))
    res = np.argmax(vector @ matrix.T)

print(res)

for key, dist, importance, words in sorted(arr, key=lambda x: x[1], reverse=True):
    print(f"{key} расстояние {dist:.4f} важные слова {importance} {set(words)}")

профильный предприятие газпром нефть внедрить грейд который позволять установить единый предприятие чёткий прозрачный принцип вознаграждение обучение оценка персонал в связь расширение география бизнес разработать внедряться единый концепция мобильный персонал кадровый база реализация крупный проект число зарубежный концепция определять порядок направление сотрудник работа регион россия граница перечень порядок предоставление компенсация льгота который позволять дополнительно мотивировать перемещать сотрудник организационный структура управление персонал включать департамент работа персонал корпоративный центр входить блок организационный вопрос управление работа персонал производственный блок подразделение работа персонал дзо этот структурный подразделение каждый свой уровень отвечать реализация стратегический программа развитие кадровый потенциал компания текущий обеспечение деятельность весь направление управление персонал
34
Вовлеченность и мотивация расстояние 0.5533 важные слова 

In [14]:
ROWS = 100_000
df = pd.read_csv("paragraphs.csv.zip")  # , usecols=["cleaned_text"], nrows=ROWS
df = df.dropna().reset_index(drop=True)
# df = df[df["cleaned_text"].apply(lambda x: len(x.split()) > 1)]
# df = df.reset_index(drop=True)
df.shape

/tmp/ipykernel_2760/610844443.py:2: DtypeWarning: Columns (0,1,2,3,4,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("paragraphs.csv.zip")  # , usecols=["cleaned_text"], nrows=ROWS


(2309485, 8)

In [16]:
topics = []

for text in tqdm(df["cleaned_text"].tolist()):
    paragraph_dict = copy.deepcopy(base_dict)
    paragraph_importance = copy.deepcopy(importance_dict)
    found_words = False
    base_words = {}
    advanced_words = {}
    vector_exp = np.zeros(matrix.shape[1])
    paragraph_len = len(text.split())

    for key, regex in re_dict.items():
        words = regex.findall(text)
        advanced_words_found = re_advanced_dict[key].findall(text)
        for word in words:
            weight = sheets_dict[key][word]
            paragraph_dict[key][word] = weight
            found_words = True
            paragraph_importance[key] += weight > 1
            vector_exp[word2idx[word]] = 1

        base_words[key] = len(words)
        advanced_words[key] = len(advanced_words_found)

    if not found_words:
        res = {key: 0 for key in sheets_dict}
        topics.append({**res, **{"max_score_cos": None, "max_topic_cos": None}})
        continue

    res = {}
    topic_found = False
    for key, vector in sheets_dict.items():
        if base_words[key] / paragraph_len < 0.1 or advanced_words[key] < 1:
            res[key] = 0
            continue
        sim = cosine_similarity(X=[list(vector.values())], Y=[list(paragraph_dict[key].values())])[0][0]
        res[key] = sim
        topic_found = True

    if not topic_found:
        res = {key: 0 for key in sheets_dict}
        topics.append({**res, **{"max_score_cos": None, "max_topic_cos": None}})
        continue

    paragraph_matrix = vector_exp @ matrix.T
    best_indicies = paragraph_matrix.argsort()[-2:][::-1]
    best_topics = [idx2label[i] for i in best_indicies]
    best_scores = paragraph_matrix[best_indicies]

    res_sorted = sorted(res.items(), key=lambda x: x[1], reverse=True)
    max_value = res_sorted[0]
    second_values = res_sorted[1]
    if second_values[1] == 0:
        second_values = (None, None)

    topics.append(
        {
            **res,
            **{
                "max_topic_cos": max_value[0],
                "max_score_cos": max_value[1],
                "second_max_topic_cos": second_values[0],
                "second_max_score_cos": second_values[1],
                "max_topic_exp": best_topics[0],
                "max_score_exp": best_scores[0],
                "second_max_topic_exp": best_topics[1],
                "second_max_score_exp": best_scores[1],
            },
        }
    )

 51%|█████     | 1171312/2309485 [7:49:39<7:36:22, 41.57it/s] 


KeyboardInterrupt: 

In [17]:
import pickle

with open("topics.pkl", "wb") as fp:  # Pickling
    pickle.dump(topics, fp)

In [18]:
topics = pd.DataFrame(topics)
topics.to_csv("all_topics.csv.zip")

In [19]:
df = df.join(topics)
df.to_csv("paragrapghs_with_topisc.csv.zip")

In [8]:
df[["max_score", "max_topic", "second_max_score", "second_max_topic"]] = df[
    ["max_topic", "max_score", "second_max_topic", "second_max_score"]
]

In [28]:
n = 30
df.groupby("max_topic_cos").sample(n, replace=True)[
    [
        "company",
        "year",
        "sector",
        "report_type",
        "paragraph",
        "original_text",
        "cleaned_text",
        "max_topic_cos",
        "max_score_cos",
        "second_max_topic_cos",
        "second_max_score_cos",
        "max_topic_exp",
        "max_score_exp",
        "second_max_topic_exp",
        "second_max_score_exp",
    ]
].to_csv(f"paragraph_max_values_{n}.csv.zip")

In [30]:
df.groupby("max_topic").first()  # ["max_score"].first()

,Unnamed: 0,company,year,sector,report_type,paragraph,original_text,cleaned_text,Климат,Энергия,...,Отчетность и прозрачность,Права человека,Лидерство,Риски,Этика,Корпоративное управление,Устойчивое развитие,max_score,second_max_score,second_max_topic
max_topic,,,,,,,,,,,,,,,,,,,,,
0.140988,2244210,АО «ТВЭЛ»,2015,Энергетика,ИО,263.0,Деятельность оказание сопутствующих услуг Пост...,деятельность оказание сопутствовать услуга пос...,0.0,0.0,...,0.000000,0,0.0,0.000000,0.0,0.0,0.0,Качество и безопасность продукт,Климат,0.000000
0.143674,1350987,ПАО «Юнипро»,2018,Энергетика,ИО,4877.0,Предмет Дополнительного соглашения: 1. Стороны...,предмет дополнительный соглашение сторона дого...,0.0,0.0,...,0.000000,0,0.0,0.000000,0.0,0.0,0.0,Качество и безопасность продукт,Климат,0.000000
0.146310,457049,ПАО «Полюс»,2017,Металлургическая и горнодобывающая,ИО,5978.0,Выбытие линий электропередачи По состоянию на ...,выбытие линия электропередача по состояние июн...,0.0,0.0,...,0.000000,0,0.0,0.000000,0.0,0.0,0.0,Качество и безопасность продукт,Климат,0.000000
0.148900,3245071,ПАО «ФСК ЕЭС» («Россети»),2009,Энергетика,ОУР,162.0,3.2. Подход и результативность работ по технич...,подход результативность работа технический обс...,0.0,0.0,...,0.000000,0,0.0,0.000000,0.0,0.0,0.0,Качество и безопасность продукт,Климат,0.000000
0.148900,1350990,ПАО «Юнипро»,2018,Энергетика,ИО,4880.0,"неполном объеме, Заказчик имеет право отказать...",неполный объём заказчик иметь право отказать п...,0.0,0.0,...,0.000000,0,0.0,0.000000,0.0,0.0,0.0,Качество и безопасность продукт,Климат,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0.845154,3765,ПАО «Ростелеком»,2019,Телекоммуникационная и связь,ОУР,2070.0,3.4. Обучение и развитие сотрудников разработк...,обучение развитие сотрудник разработка отдельн...,0.0,0.0,...,0.000000,0,0.0,0.000000,0.0,0.0,0.0,Вовлеченность и мотивация,Удовлетворенность потребителей,0.493352
0.847566,1323153,АО «СУЭК»,"2019, 2018",Металлургическая и горнодобывающая,ОУР,1621.0,Важнейшим средством продвижения корпоративной ...,важный средство продвижение корпоративный куль...,0.0,0.0,...,0.000000,0,0.0,0.000000,0.0,0.0,0.0,Вовлеченность и мотивация,Здоровье и благополучие сотрудн,0.571662
0.849970,2947503,Группа ЛСР,2020,Строительство,ОУР,94.0,Работа по усилению практик устойчивого развити...,работа усиление практика устойчивый развитие п...,0.0,0.0,...,0.391381,0,0.0,0.345949,0.0,0.0,0.0,Вовлеченность и мотивация,Обучение и развитие,0.575212


In [ ]:
df.to_csv("paragrapghs_with_topisc.csv")

In [44]:
grouped = pd.DataFrame(df.groupby(["company", "year", "max_topic"])["cleaned_text"].agg(" ".join))
grouped.head()

cleaned_text
company     year max_topic                                                                     
Alcoa, Inc. 2004 Антикоррупция                москва белый калитва восток московский офис са...
                 Безопасность и охрана труда          доход европа процент общий доход компания
                 Безопасность продукта        я очень приятно представить ваш внимание первы...
                 Благотворительность          лос летучий органический соединение в тысяча м...
                 Налоги                       возрастание выброс происходить связь применени...

In [45]:
from collections import defaultdict

topics_df = pd.concat([e_topics, s_topics, g_topics])
len_topics = defaultdict(dict)

for idx, row in tqdm(topics_df.iterrows(), total=topics_df.shape[0]):
    if not len_topics[row["topic"]].get(row["type"], False):
        len_topics[row["topic"]][row["type"]] = 0
    len_topics[row["topic"]][row["type"]] += 1

100%|██████████| 4414/4414 [00:00<00:00, 19458.61it/s]


In [46]:
len_topics

defaultdict(dict,
            {'Экология': {1: 51, 2: 8},
             'Климат': {1: 95, 2: 24},
             'Энергия': {1: 52, 2: 11},
             'Воздух': {1: 42},
             'Вода': {1: 61, 2: 8},
             'Отходы': {1: 41, 2: 10},
             'Биоразнообразие': {1: 47},
             'Социально-культурное воздействи': {1: 3},
             'Антикоррупция': {1: 50},
             'Дивиденды и акционеры': {1: 306},
             'Инновации': {1: 104},
             'Лидерство': {1: 371},
             'Отчетность и прозрачность': {1: 530},
             'Предотвращение коррупции': {1: 92},
             'Управление рисками': {1: 214},
             'Эффективность и производительность': {1: 319},
             'Безопасность и охрана труда': {1: 349},
             'Безопасность продукта': {1: 50},
             'Благотворительность': {1: 259},
             'Инвестиции и капитальные вложения': {1: 370},
             'Налоги': {1: 81},
             'Обучение и развитие': {1: 50},
        

In [47]:
from __future__ import annotations


def get_score(words: dict[str, int], topic: str) -> int:
    count_words = {key: 0 for key in len_topics[topic]}
    for word in words:
        count_words[sheets_dict[topic][word]] += 1
    base_percent = count_words[1] / len_topics[topic][1]
    advanced_percent = count_words[2] / len_topics[topic][2] if count_words.get(2, False) else 0
    if base_percent < 0.20:
        return 1
    elif base_percent < 0.40:
        return 2
    elif base_percent < 0.60:
        return 3
    elif base_percent < 0.80:
        return 4
    elif base_percent <= 1 and advanced_percent == 0:
        return 5
    elif base_percent <= 1 and advanced_percent < 0.40:
        return 6
    elif base_percent <= 1 and advanced_percent < 0.80:
        return 7
    elif base_percent <= 1 and advanced_percent <= 1:
        return 7
    else:
        return 0

In [55]:
scores = []

for (company, year, topic), row in tqdm(grouped.iterrows(), total=grouped.shape[0]):
    paragraph_dict = copy.deepcopy(base_dict)[topic]
    text = row["cleaned_text"]
    words = re_dict[topic].findall(text)
    for word in words:
        paragraph_dict[word] = 1
    scores.append(get_score(paragraph_dict, topic))

100%|██████████| 8788/8788 [01:05<00:00, 135.19it/s]


In [57]:
grouped["score"] = scores
grouped.head(20)

cleaned_text  \
company     year       max_topic                                                                              
Alcoa, Inc. 2004       Антикоррупция                      москва белый калитва восток московский офис са...   
                       Безопасность и охрана труда                доход европа процент общий доход компания   
                       Безопасность продукта              я очень приятно представить ваш внимание первы...   
                       Благотворительность                лос летучий органический соединение в тысяча м...   
                       Налоги                             возрастание выброс происходить связь применени...   
                       Отношения с потребителями                 белый калитва любучан восток москва самара   
                       Отношения с работниками            вступление выброс загрязнять вещество атмосфер...   
                       Отходы                             показатель здоровье работник безопасность труд...   
                       Предотвращение коррупции           наш преимущество заключаться производить очень...   
                       Профсоюзы и коллективные договоры  показатель экологический безопасность для комп...   
                       Управление рисками                 отчёт устойчивый развитие год отчёт ход выполн...   
BIOCAD      2019, 2018 Антикоррупция                      соответствие отчёт руководство прямой речь пре...   
                       Безопасность и охрана труда        экопроект павел яко вл быть в ранний разработк...   
                       Безопасность продукта              содержание устойчивый развитие стратегия приор...   
                       Благотворительность                август россия мир уровень вовлечённость сотруд...   
                       Вода                               характер собственность форма существенный изме...   
                       Дивиденды и акционеры              сокращать отход производство упаковка снижать ...   
                       Инвестиции и капитальные вложения  самый продавать препарат одержать победа суд п...   
                       Инновации                          генферон® ацеллбия® альгерон® эфлейра® млн руб...   
                       Климат                             о компания отход удаться ускорить синтезирован...   

                                                          score  
company     year       max_topic                                 
Alcoa, Inc. 2004       Антикоррупция                          5  
                       Безопасность и охрана труда            5  
                       Безопасность продукта                  5  
                       Благотворительность                    5  
                       Налоги                                 5  
                       Отношения с потребителями              5  
                       Отношения с работниками                5  
                       Отходы                                 7  
                       Предотвращение коррупции               5  
                       Профсоюзы и коллективные договоры      5  
                       Управление рисками                     5  
BIOCAD      2019, 2018 Антикоррупция                          5  
                       Безопасность и охрана труда            5  
                       Безопасность продукта                  5  
                       Благотворительность                    5  
                       Вода                                   7  
                       Дивиденды и акционеры                  5  
                       Инвестиции и капитальные вложения      5  
                       Инновации                              5  
                       Климат                                 7

In [58]:
grouped.to_csv("grouped_paragraphs.csv.zip")